# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key = g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
hmap_data = pd.read_csv("cities.csv")
del hmap_data["Unnamed: 0"]
hmap_data


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed
0,Busselton,100,AU,1603804086,67,-33.65,115.33,18.33,4.28
1,Narón,20,ES,1603803573,82,43.52,-8.15,17.00,11.80
2,Nakhon Phanom,40,TH,1603803574,94,17.33,104.42,26.00,0.50
3,Saint-Philippe,75,RE,1603803574,61,-21.36,55.77,27.00,4.10
4,Butaritari,100,KI,1603803574,68,3.07,172.79,27.28,3.72
...,...,...,...,...,...,...,...,...,...
533,Chumikan,100,RU,1603803724,77,54.72,135.31,3.31,6.07
534,Ixtapa,5,MX,1603803974,94,20.70,-105.20,25.56,1.43
535,Waddān,0,LY,1603803975,15,29.16,16.14,32.53,7.69
536,Taixing,91,CN,1603803724,80,32.17,120.01,18.33,3.38


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
lat = hmap_data["Latitude"]
lng = hmap_data["Longitude"]
hum = hmap_data["Humidity"].tolist()
coordinates = list(zip(lat,lng))
coordinates
hum

[67,
 82,
 94,
 61,
 68,
 63,
 20,
 61,
 84,
 87,
 83,
 92,
 79,
 78,
 79,
 67,
 54,
 44,
 95,
 78,
 74,
 96,
 77,
 79,
 64,
 72,
 81,
 29,
 84,
 70,
 11,
 90,
 100,
 100,
 37,
 66,
 73,
 65,
 88,
 50,
 39,
 79,
 17,
 89,
 92,
 72,
 98,
 96,
 61,
 78,
 89,
 60,
 77,
 83,
 100,
 100,
 68,
 91,
 97,
 75,
 91,
 89,
 27,
 85,
 82,
 64,
 18,
 88,
 81,
 66,
 100,
 100,
 59,
 28,
 65,
 90,
 93,
 86,
 91,
 56,
 85,
 93,
 86,
 86,
 73,
 68,
 93,
 23,
 80,
 43,
 85,
 79,
 90,
 35,
 89,
 83,
 60,
 92,
 82,
 68,
 98,
 93,
 72,
 80,
 74,
 65,
 74,
 93,
 92,
 63,
 15,
 76,
 93,
 61,
 82,
 66,
 82,
 74,
 52,
 88,
 72,
 51,
 92,
 93,
 74,
 71,
 100,
 60,
 74,
 77,
 38,
 64,
 34,
 74,
 54,
 78,
 100,
 81,
 96,
 90,
 85,
 14,
 100,
 56,
 73,
 94,
 100,
 71,
 56,
 83,
 94,
 66,
 40,
 78,
 79,
 69,
 45,
 80,
 40,
 71,
 94,
 62,
 65,
 81,
 10,
 93,
 90,
 87,
 30,
 93,
 91,
 93,
 51,
 69,
 92,
 60,
 68,
 79,
 91,
 50,
 75,
 50,
 66,
 95,
 87,
 76,
 88,
 79,
 96,
 75,
 14,
 82,
 69,
 88,
 93,
 88,
 56,
 82,


In [42]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout = figure_layout)
heat = gmaps.heatmap_layer(coordinates, weights=hum)
fig.add_layer(heat)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [13]:
#use loc to find ideal weather conditions
ideal_temp_b = hmap_data.loc[hmap_data["Max Temp"] < 28]
ideal_temp_b
#ideal_temp_t = ideal_temp_b.loc[ideal_temp_b["Max Temp"] < 30]
#ideal_temp_t
ideal_hum_t = ideal_temp_b.loc[ideal_temp_b["Humidity"] < 55]
#ideal_hum_t
ideal_hum_b = ideal_hum_t.loc[ideal_hum_t["Humidity"] > 45]
#ideal_hum_b
hotel_df = ideal_hum_b.loc[ideal_hum_b["Cloudiness"] < 50]

hotel_df.shape

(8, 9)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
hotel_df["Hotel Name"] = ""
#cities_pd["Lat"] =""
#cities_pd["Lng"] = ""
#cities_pd["Airport Name"] = ""
#cities_pd["Airport Address"] = ""
#cities_pd["Airport Rating"] = ""
hotel_df.head()

<ipython-input-14-e77db4689c30>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""


,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
118,Ahmednagar,26,IN,1603803600,52,19.08,74.73,24.92,3.13,
121,Mar del Plata,0,AR,1603803884,51,-38.00,-57.56,16.00,4.60,
206,Karakol,0,KG,1603803634,54,42.49,78.39,6.59,2.30,
296,Balkhash,0,KZ,1603803663,48,46.84,74.98,7.00,12.00,
333,Arys,40,KZ,1603803673,47,42.43,68.80,13.00,4.00,


In [41]:
lth = hotel_df["Latitude"]
lgh = hotel_df["Longitude"]
location = list(zip(lth,lgh))
location


[(19.08, 74.73),
 (-38.0, -57.56),
 (42.49, 78.39),
 (46.84, 74.98),
 (42.43, 68.8),
 (-32.32, -58.08),
 (-19.58, -65.75),
 (-38.37, -60.28)]

In [32]:
#Create function to iterate through my data frame
def getHotel(Latitude,Longitude):
    params = {
        "keyword": "hotel",
        "radius": 5000,
        "key": g_key,
        "location": f"{Latitude},{Longitude}"
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response_it = []

    response = requests.get(base_url, params=params).json()
        
    return response

In [34]:
#function test, will comment to avoid unnecessary calls to api, but it works
#function_response = getHotel(-38.0, -57.56)
#function_response['results'][0]['name']
#'NH Gran Hotel Provincial'


In [39]:
for i, row in hotel_df.iterrows():
    function_response = getHotel(row["Latitude"], row["Longitude"])
    hotel_df.loc[i, "Hotel Name"] = function_response['results'][0]['name']

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [40]:
hotel_df

,City,Cloudiness,Country,Date,Humidity,Latitude,Longitude,Max Temp,Wind Speed,Hotel Name
118,Ahmednagar,26,IN,1603803600,52,19.08,74.73,24.92,3.13,Hotel Gulmohar Pride
121,Mar del Plata,0,AR,1603803884,51,-38.00,-57.56,16.00,4.60,NH Gran Hotel Provincial
206,Karakol,0,KG,1603803634,54,42.49,78.39,6.59,2.30,Hotel Hillside Karakol
296,Balkhash,0,KZ,1603803663,48,46.84,74.98,7.00,12.00,Семейная зона отдыха LAFA
333,Arys,40,KZ,1603803673,47,42.43,68.80,13.00,4.00,Гостиница МАДАНИ
371,Paysandu,0,UY,1603803683,46,-32.32,-58.08,17.78,0.89,Gran Hotel Paysandú
419,Potosí,18,BO,1603803696,48,-19.58,-65.75,10.07,2.52,Hotel Boulevard
522,Tres Arroyos,0,AR,1603803972,50,-38.37,-60.28,19.12,6.13,Hotel Alfil Express


In [48]:
location

[(19.08, 74.73),
 (-38.0, -57.56),
 (42.49, 78.39),
 (46.84, 74.98),
 (42.43, 68.8),
 (-32.32, -58.08),
 (-19.58, -65.75),
 (-38.37, -60.28)]

In [49]:
fig = gmaps.figure(layout = figure_layout)
markers = gmaps.marker_layer(location)
fig.add_layer(heat)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [52]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout = figure_layout)
markers = gmaps.marker_layer(location)
fig.add_layer(heat)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…